# 📊 Modul Pembelajaran: Metode Konversi Data dalam Data Science

## 🎯 **Tujuan Pembelajaran**
Memahami dan mengimplementasikan tiga metode utama konversi data yang sering digunakan dalam preprocessing data untuk machine learning dan data analysis.

## 📚 **Materi Konversi Data**

### 🗂️ **1. Binning/Diskretisasi**
**Definisi**: Proses mengubah data kontinyu menjadi data kategorial dengan membagi nilai ke dalam interval-interval tertentu.

**Kapan Digunakan:**
- Data kontinyu yang memiliki rentang nilai yang luas
- Ingin mengurangi kompleksitas model
- Mengatasi outlier atau noise dalam data
- Membuat data lebih interpretable

**Contoh**: Usia (17-90 tahun) → Kelompok usia (Muda, Dewasa, Senior)

### 🏷️ **2. Label Encoding** 
**Definisi**: Mengubah data kategori ordinal menjadi nilai numerik berurutan yang mempertahankan hierarki/urutan.

**Kapan Digunakan:**
- Data kategori yang memiliki urutan logis (ordinal)
- Model machine learning membutuhkan input numerik
- Ingin mempertahankan informasi urutan

**Contoh**: Pendidikan (SD → SMP → SMA → S1 → S2 → S3) menjadi (0, 1, 2, 3, 4, 5)

### 🔥 **3. One-Hot Encoding**
**Definisi**: Mengubah data kategori nominal menjadi beberapa kolom binary (0 dan 1), dimana setiap kategori direpresentasikan oleh satu kolom.

**Kapan Digunakan:**
- Data kategori tanpa urutan logis (nominal)
- Mencegah model mengasumsikan urutan yang tidak ada
- Algoritma yang sensitif terhadap magnitude nilai

**Contoh**: Jenis Kelamin (Male, Female) → sex_Male (0/1), sex_Female (0/1)

## ⚖️ **Perbandingan Metode**

| Metode | Jenis Data | Input | Output | Kegunaan Utama |
|--------|------------|--------|--------|----------------|
| **Binning** | Kontinyu | Numerik | Kategorial | Simplifikasi, reduce noise |
| **Label Encoding** | Ordinal | Kategori berurutan | Numerik berurutan | Pertahankan hierarki |
| **One-Hot Encoding** | Nominal | Kategori setara | Multiple binary columns | Hindari asumsi urutan |

## 🎓 **Kasus Studi**
Dalam notebook ini, kita akan menggunakan **Adult Census Income Dataset** untuk mempraktikkan ketiga metode konversi data:
- **Age** (kontinyu) → Binning menjadi kelompok usia
- **Education** (ordinal) → Label encoding berdasarkan tingkat pendidikan  
- **Sex** (nominal) → One-hot encoding untuk jenis kelamin

---


## 🔧 1. Import Libraries dan Install Dependencies

In [1]:
# Install libraries jika belum ada
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
        print(f"✅ {package} sudah terinstall")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
packages = ['pandas', 'numpy', 'scikit-learn']
for package in packages:
    install_package(package)

# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import fetch_openml
import warnings
warnings.filterwarnings('ignore')

print("\n🎉 Semua libraries berhasil diimport!")

✅ pandas sudah terinstall
✅ numpy sudah terinstall
📦 Installing scikit-learn...

🎉 Semua libraries berhasil diimport!

🎉 Semua libraries berhasil diimport!


## 📊 2. Load Dataset dan Eksplorasi Data

Kita akan menggunakan **Adult Census Income Dataset** yang memiliki berbagai jenis data:
- **Data Kontinyu**: age, hours-per-week
- **Data Ordinal**: education (tingkatan pendidikan)
- **Data Nominal**: sex, race, marital-status

In [2]:
# Load Adult Census Income Dataset
try:
    adult_data = fetch_openml('adult', version=2, as_frame=True, parser='auto')
    df = adult_data.data.copy()
    print("✅ Dataset berhasil dimuat dari OpenML")
except:
    # Fallback: buat dataset sintetis
    print("📁 Membuat dataset sintetis...")
    np.random.seed(42)
    n_samples = 1000
    
    df = pd.DataFrame({
        'age': np.random.randint(17, 80, n_samples),
        'sex': np.random.choice(['Male', 'Female'], n_samples),
        'education': np.random.choice(['HS-grad', 'Some-college', 'Bachelors', 'Masters', 'Doctorate'], n_samples),
        'hours-per-week': np.random.randint(1, 80, n_samples),
        'race': np.random.choice(['White', 'Black', 'Asian-Pac-Islander'], n_samples),
        'marital-status': np.random.choice(['Married-civ-spouse', 'Never-married', 'Divorced'], n_samples)
    })

print(f"\n📏 Dataset shape: {df.shape}")
print(f"📋 Kolom tersedia: {list(df.columns)}")

# Tampilkan sample data
print("\n📄 Sample Data (5 baris pertama):")
print(df.head())

# Info dataset
print("\n📊 Info Dataset:")
print(df.info())

✅ Dataset berhasil dimuat dari OpenML

📏 Dataset shape: (48842, 14)
📋 Kolom tersedia: ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']

📄 Sample Data (5 baris pertama):
   age  workclass  fnlwgt     education  education-num      marital-status  \
0   25    Private  226802          11th              7       Never-married   
1   38    Private   89814       HS-grad              9  Married-civ-spouse   
2   28  Local-gov  336951    Assoc-acdm             12  Married-civ-spouse   
3   44    Private  160323  Some-college             10  Married-civ-spouse   
4   18        NaN  103497  Some-college             10       Never-married   

          occupation relationship   race     sex  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black    Male             0             0   
1    Farming-fishing      Husband  White    Male           

## 🎯 3. Pemilihan Kolom untuk Konversi

Berdasarkan jenis data, kita akan memilih 3 kolom:
1. **Data Kontinyu**: `age` - untuk Binning/Diskretisasi
2. **Data Ordinal**: `education` - untuk Label Encoding
3. **Data Nominal**: `sex` - untuk One-Hot Encoding

Mari kita pastikan tidak ada missing data pada kolom-kolom ini.

In [3]:
# Pilih kolom untuk konversi
selected_columns = {
    'kontinyu': 'age',      # Untuk Binning
    'ordinal': 'education', # Untuk Label Encoding  
    'nominal': 'sex'        # Untuk One-Hot Encoding
}

print("🔍 Analisis Kolom Terpilih:")
print("=" * 40)

for jenis, kolom in selected_columns.items():
    print(f"\n{jenis.upper()} - {kolom}:")
    print(f"  Missing values: {df[kolom].isnull().sum()}")
    print(f"  Unique values: {df[kolom].nunique()}")
    print(f"  Sample values: {list(df[kolom].unique()[:5])}")

# Buat dataframe kerja dengan kolom terpilih
df_work = df[list(selected_columns.values())].copy()

# Pastikan tidak ada missing data
print(f"\n✅ Dataset kerja shape: {df_work.shape}")
print(f"📊 Missing data check:")
print(df_work.isnull().sum())

if df_work.isnull().sum().sum() > 0:
    print("⚠️ Ada missing data, akan dihapus...")
    df_work = df_work.dropna()
    print(f"📏 Shape setelah hapus missing: {df_work.shape}")

print(f"\n📋 Dataset Kerja:")
print(df_work.head())

🔍 Analisis Kolom Terpilih:

KONTINYU - age:
  Missing values: 0
  Unique values: 74
  Sample values: [np.int64(25), np.int64(38), np.int64(28), np.int64(44), np.int64(18)]

ORDINAL - education:
  Missing values: 0
  Unique values: 16
  Sample values: ['11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th']

NOMINAL - sex:
  Missing values: 0
  Unique values: 2
  Sample values: ['Male', 'Female']

✅ Dataset kerja shape: (48842, 3)
📊 Missing data check:
age          0
education    0
sex          0
dtype: int64

📋 Dataset Kerja:
   age     education     sex
0   25          11th    Male
1   38       HS-grad    Male
2   28    Assoc-acdm    Male
3   44  Some-college    Male
4   18  Some-college  Female


## 🗂️ 4. Implementasi Binning/Diskretisasi

**Binning** adalah proses mengubah data kontinyu menjadi data kategorial dengan membagi nilai ke dalam interval tertentu.

**Kegunaan:**
- Mengurangi noise dalam data
- Membuat data lebih interpretable
- Mengatasi outlier

Kita akan menerapkan binning pada kolom `age` untuk membuat kelompok usia.

In [4]:
# Implementasi Binning pada kolom 'age'
print("🗂️ BINNING/DISKRETISASI - Kolom 'age'")
print("=" * 40)

# Tampilkan statistik age sebelum binning
print(f"📊 Statistik Age:")
print(f"  Min: {df_work['age'].min()}")
print(f"  Max: {df_work['age'].max()}")
print(f"  Mean: {df_work['age'].mean():.1f}")

# Buat bins berdasarkan tahap kehidupan
age_bins = [0, 25, 35, 50, 65, 100]
age_labels = ['Muda (17-25)', 'Dewasa Muda (26-35)', 'Dewasa (36-50)', 'Paruh Baya (51-65)', 'Senior (65+)']

# Implementasi binning
df_work['age_binned'] = pd.cut(df_work['age'], 
                              bins=age_bins, 
                              labels=age_labels, 
                              include_lowest=True)

print(f"\n✅ Binning selesai!")
print(f"📋 Distribusi Age setelah Binning:")
print(df_work['age_binned'].value_counts().sort_index())

# Tampilkan contoh transformasi
print(f"\n📄 Contoh Transformasi:")
sample_data = df_work[['age', 'age_binned']].head(10)
print(sample_data)

🗂️ BINNING/DISKRETISASI - Kolom 'age'
📊 Statistik Age:
  Min: 17
  Max: 90
  Mean: 38.6

✅ Binning selesai!
📋 Distribusi Age setelah Binning:
age_binned
Muda (17-25)            9627
Dewasa Muda (26-35)    12719
Dewasa (36-50)         16688
Paruh Baya (51-65)      8005
Senior (65+)            1803
Name: count, dtype: int64

📄 Contoh Transformasi:
   age           age_binned
0   25         Muda (17-25)
1   38       Dewasa (36-50)
2   28  Dewasa Muda (26-35)
3   44       Dewasa (36-50)
4   18         Muda (17-25)
5   34  Dewasa Muda (26-35)
6   29  Dewasa Muda (26-35)
7   63   Paruh Baya (51-65)
8   24         Muda (17-25)
9   55   Paruh Baya (51-65)


## 🏷️ 5. Implementasi Label Encoding

**Label Encoding** mengubah data kategori ordinal menjadi angka berurutan yang mempertahankan urutan/hierarki.

**Karakteristik Data Ordinal:**
- Ada urutan yang bermakna
- Contoh: tingkat pendidikan (SD < SMP < SMA < S1 < S2 < S3)

**Penting:** Harus mendefinisikan urutan yang benar, jangan gunakan encoding otomatis yang mengurutkan secara alfabetis!

In [5]:
# Implementasi Label Encoding pada kolom 'education'
print("🏷️ LABEL ENCODING - Kolom 'education'")
print("=" * 40)

# Tampilkan kategori education yang ada
print(f"📋 Kategori Education yang ada:")
education_counts = df_work['education'].value_counts()
print(education_counts)

# Definisikan urutan education yang benar (dari rendah ke tinggi)
education_order = [
    'Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th',
    'HS-grad', 'Some-college', 'Assoc-voc', 'Assoc-acdm', 
    'Bachelors', 'Masters', 'Prof-school', 'Doctorate'
]

# Filter hanya education yang ada di data
available_education = [edu for edu in education_order if edu in df_work['education'].values]
print(f"\n📚 Urutan Education yang tersedia:")
for i, edu in enumerate(available_education):
    print(f"  {i}: {edu}")

# Manual mapping dengan urutan yang benar
education_mapping = {edu: idx for idx, edu in enumerate(available_education)}

# Implementasi Label Encoding
df_work['education_encoded'] = df_work['education'].map(education_mapping)

print(f"\n✅ Label Encoding selesai!")
print(f"📋 Mapping Education:")
for edu, code in education_mapping.items():
    count = (df_work['education'] == edu).sum()
    print(f"  {code}: {edu} ({count} data)")

# Tampilkan contoh transformasi
print(f"\n📄 Contoh Transformasi:")
sample_data = df_work[['education', 'education_encoded']].head(10)
print(sample_data)

🏷️ LABEL ENCODING - Kolom 'education'
📋 Kategori Education yang ada:
education
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: count, dtype: int64

📚 Urutan Education yang tersedia:
  0: Preschool
  1: 1st-4th
  2: 5th-6th
  3: 7th-8th
  4: 9th
  5: 10th
  6: 11th
  7: 12th
  8: HS-grad
  9: Some-college
  10: Assoc-voc
  11: Assoc-acdm
  12: Bachelors
  13: Masters
  14: Prof-school
  15: Doctorate

✅ Label Encoding selesai!
📋 Mapping Education:
  0: Preschool (83 data)
  1: 1st-4th (247 data)
  2: 5th-6th (509 data)
  3: 7th-8th (955 data)
  4: 9th (756 data)
  5: 10th (1389 data)
  6: 11th (1812 data)
  7: 12th (657 data)
  8: HS-grad (15784 data)
  9: Some-college (10878 data)


## 🔥 6. Implementasi One-Hot Encoding

**One-Hot Encoding** mengubah data kategori nominal menjadi beberapa kolom binary (0 dan 1).

**Karakteristik Data Nominal:**
- Tidak ada urutan yang bermakna
- Setiap kategori setara
- Contoh: jenis kelamin, warna, negara

**Keuntungan:** Tidak ada asumsi urutan, setiap kategori diperlakukan adil  
**Kerugian:** Meningkatkan dimensi data (1 kolom → n kolom)

In [6]:
# Implementasi One-Hot Encoding pada kolom 'sex'
print("🔥 ONE-HOT ENCODING - Kolom 'sex'")
print("=" * 40)

# Tampilkan kategori sex yang ada
print(f"📋 Kategori Sex yang ada:")
sex_counts = df_work['sex'].value_counts()
print(sex_counts)

# Implementasi One-Hot Encoding menggunakan pandas
sex_onehot = pd.get_dummies(df_work['sex'], prefix='sex')

print(f"\n✅ One-Hot Encoding selesai!")
print(f"📊 Hasil transformasi:")
print(f"  Kolom asli: 1 kolom ('sex')")
print(f"  Kolom baru: {sex_onehot.shape[1]} kolom")
print(f"  Kolom yang dibuat: {list(sex_onehot.columns)}")

# Gabungkan dengan dataframe kerja
df_work = pd.concat([df_work, sex_onehot], axis=1)

print(f"\n📄 Contoh Transformasi:")
sample_data = df_work[['sex'] + list(sex_onehot.columns)].head(10)
print(sample_data)

# Verifikasi: setiap baris harus memiliki tepat satu nilai 1
print(f"\n🔍 Verifikasi One-Hot Encoding:")
print(f"Setiap baris harus sum = 1:")
row_sums = sex_onehot.sum(axis=1)
print(f"Min sum: {row_sums.min()}, Max sum: {row_sums.max()}")
print("✅ Verifikasi berhasil!" if row_sums.min() == row_sums.max() == 1 else "❌ Ada masalah!")

🔥 ONE-HOT ENCODING - Kolom 'sex'
📋 Kategori Sex yang ada:
sex
Male      32650
Female    16192
Name: count, dtype: int64

✅ One-Hot Encoding selesai!
📊 Hasil transformasi:
  Kolom asli: 1 kolom ('sex')
  Kolom baru: 2 kolom
  Kolom yang dibuat: ['sex_Female', 'sex_Male']

📄 Contoh Transformasi:
      sex  sex_Female  sex_Male
0    Male       False      True
1    Male       False      True
2    Male       False      True
3    Male       False      True
4  Female        True     False
5    Male       False      True
6    Male       False      True
7    Male       False      True
8  Female        True     False
9    Male       False      True

🔍 Verifikasi One-Hot Encoding:
Setiap baris harus sum = 1:
Min sum: 1, Max sum: 1
✅ Verifikasi berhasil!


## 📊 7. Evaluasi dan Gabungan Hasil

Mari kita evaluasi hasil konversi data dan buat dataframe final yang menunjukkan perbandingan sebelum dan sesudah konversi.

**Ringkasan Metode:**
- **Binning**: age (kontinyu) → age_binned (kategorial)
- **Label Encoding**: education (ordinal) → education_encoded (numerik)
- **One-Hot Encoding**: sex (nominal) → sex_Female, sex_Male (binary)

In [7]:
# Evaluasi dan Gabungan Hasil Konversi Data
print("EVALUASI HASIL KONVERSI DATA")
print("=" * 50)

# Buat dataframe final dengan kolom sebelum dan sesudah konversi berurutan
df_final = pd.DataFrame()

# 1. Age: Sebelum (kontinyu) dan Sesudah (binned)
df_final['age_original'] = df_work['age']
df_final['age_binned'] = df_work['age_binned'].astype(str)  # Convert to string for consistency

# 2. Education: Sebelum (kategori) dan Sesudah (encoded)
df_final['education_original'] = df_work['education'] 
df_final['education_encoded'] = df_work['education_encoded'].astype(int)  # Ensure it's integer

# 3. Sex: Sebelum (nominal) dan Sesudah (one-hot)
df_final['sex_original'] = df_work['sex']
onehot_cols = [col for col in df_work.columns if col.startswith('sex_')]
for col in onehot_cols:
    df_final[col] = df_work[col].astype(int)  # Convert boolean to int

print("Dataset final berhasil dibuat!")
print(f"Shape dataset final: {df_final.shape}")
print(f"Kolom: {list(df_final.columns)}")

# Verifikasi urutan education
print(f"\nVERIFIKASI URUTAN EDUCATION:")
print("-" * 40)
print("Urutan yang didefinisikan:")
for i, edu in enumerate(available_education):
    count = (df_final['education_original'] == edu).sum()
    if count > 0:
        print(f"  {i}: {edu} ({count} data)")

# Cek apakah urutan sudah benar
education_unique = df_final['education_original'].unique()
print(f"\nEducation yang ada di data: {len(education_unique)} kategori")
print("Urutan berdasarkan tingkat pendidikan (rendah ke tinggi):")
print("Urutan sudah benar dari Preschool sampai Doctorate")

# Tampilkan ringkasan konversi
print(f"\nRINGKASAN KONVERSI:")
print("-" * 30)
print(f"1. BINNING:")
print(f"   age_original (kontinyu) -> age_binned (kategorial)")
print(f"   Range: {df_final['age_original'].min()}-{df_final['age_original'].max()} -> {df_final['age_binned'].nunique()} kategori")

print(f"\n2. LABEL ENCODING:")
print(f"   education_original (ordinal) -> education_encoded (numerik)")
print(f"   Kategori: {df_final['education_original'].nunique()} -> Range 0-{df_final['education_encoded'].max()}")

print(f"\n3. ONE-HOT ENCODING:")
print(f"   sex_original (nominal) -> {len(onehot_cols)} kolom binary")
print(f"   Kategori: {df_final['sex_original'].nunique()} -> {len(onehot_cols)} kolom (0/1)")

# Tampilkan dataframe dalam format tabel yang rapi
print(f"\nTABEL HASIL KONVERSI DATA (100 baris pertama)")
print("=" * 100)

# Set pandas display options untuk tabel yang lebih rapi
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 25)
pd.set_option('display.precision', 0)

# Tampilkan tabel dengan format yang rapi
print("PERBANDINGAN SEBELUM DAN SESUDAH KONVERSI:")
print("-" * 100)

# Tampilkan 100 baris dalam format tabel
display_df = df_final.head(100).copy()

# Format display untuk lebih rapi
print(display_df.to_string(index=True, 
                          col_space=15,
                          justify='left'))

# Reset display options
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns') 
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')
pd.reset_option('display.precision')

print(f"\nKETERANGAN KOLOM:")
print("-" * 50)
print("- age_original: Usia asli (17-90 tahun)")
print("- age_binned: Kelompok usia hasil binning")  
print("- education_original: Tingkat pendidikan asli")
print("- education_encoded: Kode numerik pendidikan (0-15)")
print("- sex_original: Jenis kelamin asli (Male/Female)")
print("- sex_Female: 1 jika Female, 0 jika Male")
print("- sex_Male: 1 jika Male, 0 jika Female")

# Tampilkan sampel mapping education untuk verifikasi
print(f"\nSAMPLE MAPPING EDUCATION (untuk verifikasi):")
print("-" * 50)
education_sample = df_final[['education_original', 'education_encoded']].drop_duplicates().sort_values('education_encoded')
print(education_sample.to_string(index=False))

# Analisis perubahan dimensi
original_cols = 3  # age, education, sex
final_cols = len(df_final.columns) - 3  # minus kolom original
print(f"\nANALISIS DIMENSI:")
print(f"   Kolom asli: {original_cols}")
print(f"   Kolom hasil konversi: {final_cols}")
print(f"   Total kolom final: {len(df_final.columns)}")
print(f"   Peningkatan: {final_cols/original_cols:.1f}x lipat")

# Cek tidak ada missing values
print(f"\nCEK KUALITAS DATA:")
print(f"   Missing values: {df_final.isnull().sum().sum()}")
print(f"   Total data: {len(df_final)} baris")

print(f"\nKONVERSI DATA BERHASIL!")
print("Semua metode telah diimplementasikan dengan benar")
print("Urutan education sudah sesuai tingkat pendidikan")
print("Tabel hasil konversi sudah ditampilkan dengan rapi")
print("Data siap untuk analisis lebih lanjut atau machine learning")

EVALUASI HASIL KONVERSI DATA
Dataset final berhasil dibuat!
Shape dataset final: (48842, 7)
Kolom: ['age_original', 'age_binned', 'education_original', 'education_encoded', 'sex_original', 'sex_Female', 'sex_Male']

VERIFIKASI URUTAN EDUCATION:
----------------------------------------
Urutan yang didefinisikan:
  0: Preschool (83 data)
  1: 1st-4th (247 data)
  2: 5th-6th (509 data)
  3: 7th-8th (955 data)
  4: 9th (756 data)
  5: 10th (1389 data)
  6: 11th (1812 data)
  7: 12th (657 data)
  8: HS-grad (15784 data)
  9: Some-college (10878 data)
  10: Assoc-voc (2061 data)
  11: Assoc-acdm (1601 data)
  12: Bachelors (8025 data)
  13: Masters (2657 data)
  14: Prof-school (834 data)
  15: Doctorate (594 data)

Education yang ada di data: 16 kategori
Urutan berdasarkan tingkat pendidikan (rendah ke tinggi):
Urutan sudah benar dari Preschool sampai Doctorate

RINGKASAN KONVERSI:
------------------------------
1. BINNING:
   age_original (kontinyu) -> age_binned (kategorial)
   Range: 17-

In [8]:
display_df

,age_original,age_binned,education_original,education_encoded,sex_original,sex_Female,sex_Male
0,25,Muda (17-25),11th,6,Male,0,1
1,38,Dewasa (36-50),HS-grad,8,Male,0,1
2,28,Dewasa Muda (26-35),Assoc-acdm,11,Male,0,1
3,44,Dewasa (36-50),Some-college,9,Male,0,1
4,18,Muda (17-25),Some-college,9,Female,1,0
...,...,...,...,...,...,...,...
95,20,Muda (17-25),HS-grad,8,Male,0,1
96,25,Muda (17-25),Bachelors,12,Female,1,0
97,49,Dewasa (36-50),10th,5,Male,0,1
98,59,Paruh Baya (51-65),HS-grad,8,Male,0,1


**👤 Author**: Ferdian Bangkit Wijaya  
**🏛️ Afiliasi**: Universitas Sultan Ageng Tirtayasa  
**📧 Contact**: ferdian.bangkit@untirta.ac.id  
**📅 Last Updated**: September 2025